# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [48]:
from langchain.agents import tool
from pydantic import BaseModel, Field
from langchain.tools import format_tool_to_openai_function

In [3]:
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [4]:
@tool(args_schema=SearchInput)
def search(query: str)-> str:
    """Search for weather online"""
    return "42f"

In [5]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [6]:
search.description

'search(query: str) -> str - Search for weather online'

In [7]:
search.run("sf")

'42f'

In [14]:
import requests
from datetime import datetime

In [40]:
class OpenMeteoInput(BaseModel):
    latitude: float = Field(description="latitude of the location")
    longitude: float = Field(description="longitude of the location")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float)-> dict:
    """Fetch current temperature for given coordinates"""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1
    }
    
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    temp = data['hourly']['temperature_2m']
    times = list(map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M"), data['hourly']['time']))
    now = datetime.now()
    times_diff = [abs((now - t).total_seconds()) for t in times]
    
    temp_sorted = sorted(zip(temp, times), key=lambda x: x[1])
    
    nearest_time_temp = temp_sorted[0][0]
    
    return nearest_time_temp

In [41]:
get_current_temperature({"latitude": 13, "longitude": 14})

23.1

In [ ]:
wikipedia.exceptions.Pa

In [82]:
import wikipedia

@tool
def search_wikipedia(query: str)-> str:
    """Run wikipedia search and get page summaries"""
    
    page_titles = wikipedia.search(query)
    
    summaries = []
    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {wiki_page.title}\nSummary: {wiki_page.summary}")
        except (
            wikipedia.exceptions.PageError,
            wikipedia.exceptions.DisambiguationError
        ):
            pass
    
    
    return '\n\n'.join(summaries)

In [46]:
search_wikipedia.name

'search_wikipedia'

In [47]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run wikipedia search and get page summaries'

In [49]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run wikipedia search and get page summaries',
 'parameters': {'title': 'search_wikipediaSchemaSchema',
  'type': 'object',
  'properties': {'query': {'title': 'Query', 'type': 'string'}},
  'required': ['query']}}

In [50]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates',
 'parameters': {'title': 'OpenMeteoInput',
  'type': 'object',
  'properties': {'latitude': {'title': 'Latitude',
    'description': 'latitude of the location',
    'type': 'number'},
   'longitude': {'title': 'Longitude',
    'description': 'longitude of the location',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

## OpenAPI Spec to OpenAI Function Definition

In [51]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [52]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [53]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [54]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [55]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [56]:
pet_callables

<function langchain.chains.openai_functions.openapi.openapi_spec_to_openai_fn.<locals>.default_call_api(name: 'str', fn_args: 'dict', headers: 'Optional[dict]' = None, params: 'Optional[dict]' = None, **kwargs: 'Any') -> 'Any'>

In [57]:
from langchain.chat_models import ChatOpenAI

In [58]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [59]:
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'listPets', 'arguments': '{\n  "params": {\n    "limit": 3\n  }\n}'}})

In [60]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'showPetById', 'arguments': '{\n  "path_params": {\n    "petId": "42"\n  }\n}'}})

## Routing

In [61]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]

model = ChatOpenAI(temperature=0).bind(functions=functions)

In [62]:
model.invoke("What's the temperature in new delhi right now?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 28.6139,\n  "longitude": 77.2090\n}'}})

In [63]:
model.invoke("In which year was Playstation 5 released?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{\n  "query": "Playstation 5"\n}'}})

In [64]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}")
])

chain = prompt | model

In [65]:
chain.invoke({"input": "What's the temperature in new delhi right now?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 28.6139,\n  "longitude": 77.2090\n}'}})

In [68]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [69]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [70]:
result = chain.invoke({"input": "What's the weather in new delhi right now?"})

In [71]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [74]:
get_current_temperature(result.tool_input)

15.5

In [75]:
result = chain.invoke({"input": "Hello!"})

In [76]:
result

AgentFinish(return_values={'output': 'Hi there! How can I assist you today?'}, log='Hi there! How can I assist you today?')

In [77]:
from langchain.schema.agent import AgentFinish

In [78]:
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values["output"]
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature
        }
        return tools[result.tool].run(result.tool_input)

In [79]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [80]:
chain.invoke({"input": "What's the weather in ghaziabad right now?"})

15.4

In [84]:
print(chain.invoke({"input": "What is Playstation 5?"}))

Page: PlayStation 5
Summary: The PlayStation 5 (PS5) is a home video game console developed by Sony Interactive Entertainment. It was announced as the successor to the PlayStation 4 in April 2019, was launched on November 12, 2020, in Australia, Japan, New Zealand, North America, and South Korea, and was released worldwide one week later. The PS5 is part of the ninth generation of video game consoles, along with Microsoft's Xbox Series X/S consoles, which were released in the same month.
The base model includes an optical disc drive compatible with Ultra HD Blu-ray discs. The Digital Edition lacks this drive, as a lower-cost model for buying games only through download. The two variants were launched simultaneously. Slimmer hardware revisions of both models replaced the original models on sale in November 2023.The PlayStation 5's main hardware features include a solid-state drive customized for high-speed data streaming to enable significant improvements in storage performance, an AMD 

In [85]:
chain.invoke({"input": "How dee do dee do?"})

'Hello! How can I assist you today?'